In [30]:
import torch
import torch.nn as nn
import torch.functional as F
import torch.optim as optim
import torchvision
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random 
import time
from tqdm import tqdm_notebook as tq
import warnings
import pickle as pkl
warnings.filterwarnings("ignore")
import string
import sys
from nltk.corpus import stopwords
plt.ion()

In [ ]:
reviews = pd.read_csv("model/master_reviews.csv")

In [ ]:
reviews.head()

In [4]:
reviews.shape

(6685899, 11)

In [5]:
reviews_mod = reviews.dropna()

In [6]:
reviews_mod.shape

(6685820, 11)

In [2]:
class SentimentAnalysis:
    def __init__(self):
        
        self.map_word = {}
        self.map_index = {}
        self.x_y_pairs = {}
        self.pairs_map = {}
        
#    def word_embed(self)
    
#     def word2vec(self):
#         self.window_size = 2
#         self.epochs = 100
#         self.embeddings = 10
#         self.vocab_size = length(sla.pairs_map[self.window_size].keys())
        
#         weight1 = torch.randn(self.embeddings,self.vocab_size)
#         weight2 = torch.randn(self.vocab_size,self.embeddings)
        
#         for epoch in epochs:
#             for center,context in 
    
    def NGRAM(self):
        pass

In [96]:

class TrainSelfEmbedding:
    
    def __init__(self):
        self.window_size = 2
        self.epochs = 100
        self.embeddings = 100
        self.epoch_loss = 0
        
    def train(self,pairs_map,epochs = 100,sample_size=10**4):
        
        self.vocab_size = len(sla.map_word.keys())
        
        print(self.vocab_size)
        
        
        weight1 = torch.randn(self.embeddings,self.vocab_size).float()
        weight2 = torch.randn(self.vocab_size,self.embeddings).float()
        
        print(sys.getsizeof(weight1.storage()),weight1.element_size()*weight1.nelement()/10**6)
        print(sys.getsizeof(weight2.storage()),weight2.element_size()*weight2.nelement()/10**6)
        
        #total_loss = 0
        
        self.epochs = epochs
        
        loss_per_epoch = []
        for i in tq(range(self.epochs)):
            
            total_loss = 0
            
            
            sampled_pairs = random.sample(pairs_map[self.window_size],sample_size)
            
            for i in tq(range(sample_size)):

                center,target = sampled_pairs[i]
                inpt = torch.zeros(self.vocab_size).float()
                inpt[center] = 1.0
                z1 = torch.matmul(weight1,inpt)
                a1 = torch.relu(z1)
                
                #print(a1.size(),z1.size())
                
                z2 = torch.matmul(weight2,z1)

                log_output = torch.log_softmax(z2,dim=0)

#                 print(inpt.size())
#                 print(weight1.size())
#                 print(z1.size())


#                 print(weight2.size())
#                 print(z2.size())
#                 print(log_output.size())

                loss = torch.nn.functional.nll_loss(log_output.view(1,-1),torch.tensor([target]))

                total_loss += loss
                
                print(loss)

            loss_per_epoch.append(total_loss/sample_size)
            
            print(total_loss)
            return
            
        return loss_per_epoch
            

In [3]:
def tokenize_vocab(map_word,map_index,x):
    
    idx = len(map_word.keys())
    ans = []
    #print(type(x),x)
    for word in x.split():
        if word not in map_word:
            idx+=1
            map_word[word] = idx
            map_index[idx] = word
        ans.append(idx)
            
    return ans


def build_context_center(x,pairs_map,window = 2):
    
    if window not in pairs_map:
        pairs_map[window] = []
        
    #x = x.split()
    for i in range(len(x)):
        
        for j in range(1,window+1):
            
            if (i-j)>0:
                pairs_map[window].append([x[i],x[i-j]])
                
            if (i+j)<len(x):
                pairs_map[window].append([x[i],x[i+j]])
                
                

In [9]:
%%time
sla = SentimentAnalysis()
reviews_mod['tokenized'] = reviews_mod.text.apply(lambda x: tokenize_vocab(sla.map_word,sla.map_index,x))

CPU times: user 2min 13s, sys: 3.12 s, total: 2min 16s
Wall time: 2min 16s


In [10]:
reviews_mod.text_length.max()

4755

In [11]:
%%time
_ = reviews_mod.iloc[:,].tokenized.apply(lambda x:build_context_center(x,sla.pairs_map))

CPU times: user 19min 8s, sys: 1min 13s, total: 20min 21s
Wall time: 20min 19s


In [12]:
sla.pairs_map[2][:10]

[[1, 2],
 [1, 3],
 [2, 3],
 [2, 4],
 [3, 2],
 [3, 4],
 [3, 5],
 [4, 3],
 [4, 5],
 [4, 2]]

In [13]:
%%time
import pickle as pkl
with open("model/map.pkl",'wb') as file:
    pkl.dump(sla.pairs_map,file)

CPU times: user 14min 46s, sys: 1min 36s, total: 16min 22s
Wall time: 18min 53s


In [14]:
%%time
with open("model/map_word.pkl",'wb') as file:
    pkl.dump(sla.map_word,file)

CPU times: user 1.53 s, sys: 241 ms, total: 1.77 s
Wall time: 2.14 s


In [15]:
%%time

with open("model/map_index.pkl",'wb') as file:
    pkl.dump(sla.map_index,file)

CPU times: user 1.45 s, sys: 225 ms, total: 1.67 s
Wall time: 1.97 s


### Get pre-cleaned data

In [4]:
%%time

sla = SentimentAnalysis()
with open("model/map.pkl",'rb') as file:
    sla.pairs_map = pkl.load(file)
    
print("pairs map import complete**********")  

with open("model/map_word.pkl",'rb') as file:
    sla.map_word = pkl.load(file)
    
print("map word import complete*************")
    
with open("model/map_index.pkl",'rb') as file:
    sla.map_index = pkl.load(file)
    
print("map index import complete*************")
    

pairs map import complete**********
map word import complete*************
map index import complete*************
CPU times: user 15min 37s, sys: 4min 19s, total: 19min 56s
Wall time: 22min


In [97]:
self_embedding = TrainSelfEmbedding()

In [101]:
%%time
loss = self_embedding.train(sla.pairs_map,epochs= 50, sample_size= 100000)

3020286
1208114472 1208.1144
1208114472 1208.1144


tensor(57.7818)
tensor(41.1581)
tensor(55.6840)
tensor(56.7426)
tensor(49.7838)
tensor(51.4614)
tensor(40.2027)
tensor(63.1794)
tensor(60.3845)
tensor(49.6701)
tensor(54.6666)
tensor(44.0248)
tensor(58.4617)
tensor(45.3009)
tensor(36.1025)
tensor(40.2093)
tensor(40.4125)
tensor(50.9698)
tensor(45.1132)
tensor(63.3402)
tensor(72.5142)
tensor(40.6595)
tensor(62.8920)
tensor(49.5524)
tensor(52.1223)
tensor(61.6918)
tensor(60.8039)
tensor(45.8491)
tensor(50.0970)
tensor(67.9536)
tensor(66.6899)
tensor(57.4532)
tensor(48.8159)
tensor(48.9852)
tensor(56.7833)
tensor(49.0942)
tensor(33.2180)
tensor(65.8243)
tensor(60.8625)
tensor(53.7525)
tensor(49.0988)
tensor(68.4568)
tensor(57.5364)
tensor(74.1951)
tensor(52.5838)
tensor(67.4170)
tensor(42.9631)
tensor(60.0529)
tensor(58.3338)
tensor(53.9319)
tensor(58.2002)
tensor(55.5558)
tensor(55.8845)
tensor(75.2874)
tensor(50.7334)
tensor(52.6333)
tensor(58.0668)
tensor(49.9611)
tensor(58.5183)
tensor(50.1018)
tensor(42.9439)
tensor(49.6366)
tensor(5

tensor(46.8874)
tensor(59.6686)
tensor(53.0755)
tensor(37.5632)
tensor(40.3399)
tensor(52.5023)
tensor(46.5642)
tensor(49.2727)
tensor(55.9752)
tensor(67.0501)
tensor(53.4722)
tensor(45.0818)
tensor(42.8525)
tensor(39.3266)
tensor(44.0729)
tensor(66.0854)
tensor(39.7643)
tensor(39.7016)
tensor(50.2658)
tensor(49.3636)
tensor(51.4088)
tensor(38.8685)
tensor(67.6378)
tensor(45.9987)
tensor(51.0771)
tensor(58.7171)
tensor(43.0808)
tensor(42.9470)
tensor(48.2764)
tensor(49.4541)
tensor(53.0791)
tensor(54.9196)
tensor(46.7344)
tensor(52.9915)
tensor(56.8030)
tensor(39.5330)
tensor(66.3898)
tensor(45.0396)
tensor(48.2623)
tensor(44.7619)
tensor(61.2039)
tensor(59.3806)
tensor(51.6480)
tensor(35.6541)
tensor(51.4254)
tensor(60.4192)
tensor(35.9149)
tensor(64.7984)
tensor(59.9071)
tensor(56.4348)
tensor(65.4823)
tensor(58.6573)
tensor(49.1421)
tensor(54.7842)
tensor(63.6264)
tensor(55.2883)
tensor(73.4872)
tensor(37.8880)
tensor(58.9423)
tensor(46.4294)
tensor(63.8239)
tensor(62.9876)
tensor(4

tensor(41.6938)
tensor(53.7201)
tensor(60.5639)
tensor(52.6336)
tensor(32.8343)
tensor(49.2612)
tensor(45.5880)
tensor(38.8516)
tensor(53.1565)
tensor(28.8631)
tensor(42.2360)
tensor(28.6803)
tensor(57.3963)
tensor(52.4324)
tensor(37.7261)
tensor(54.8168)
tensor(62.0556)
tensor(37.7019)
tensor(60.9601)
tensor(38.7221)
tensor(54.1923)
tensor(24.0150)
tensor(57.8174)
tensor(47.5078)
tensor(39.9528)
tensor(46.0092)
tensor(48.6489)
tensor(41.4914)
tensor(41.5625)
tensor(51.7044)
tensor(49.5873)
tensor(48.2811)
tensor(42.7127)
tensor(47.6436)
tensor(51.0273)
tensor(39.9364)
tensor(33.9354)
tensor(39.2298)
tensor(51.9965)
tensor(21.5294)
tensor(52.7861)
tensor(32.3321)
tensor(62.4121)
tensor(49.2610)
tensor(69.3771)
tensor(41.6023)
tensor(54.4530)
tensor(45.1272)
tensor(62.9786)
tensor(73.9215)
tensor(56.3489)
tensor(40.2224)
tensor(72.8024)
tensor(47.5049)
tensor(52.8741)
tensor(57.6691)
tensor(59.8557)
tensor(56.1281)
tensor(68.4857)
tensor(61.6646)
tensor(52.1268)
tensor(61.2397)
tensor(5

tensor(57.5322)
tensor(59.1160)
tensor(47.4430)
tensor(54.7247)
tensor(40.8450)
tensor(33.3891)
tensor(45.9410)
tensor(60.3081)
tensor(60.3658)
tensor(54.6993)
tensor(41.0961)
tensor(54.8856)
tensor(71.9716)
tensor(39.8507)
tensor(76.3377)
tensor(39.6661)
tensor(29.4909)
tensor(44.4290)
tensor(70.6173)
tensor(49.7109)
tensor(38.3875)
tensor(29.6151)
tensor(61.9292)
tensor(52.8426)
tensor(31.9542)
tensor(36.7282)
tensor(48.5124)
tensor(39.6151)
tensor(29.8539)
tensor(35.5688)
tensor(59.0385)
tensor(34.1507)
tensor(44.5494)
tensor(49.0469)
tensor(46.7050)
tensor(23.3899)
tensor(48.3484)
tensor(47.1248)
tensor(66.5454)
tensor(54.3197)
tensor(55.1383)
tensor(54.5917)
tensor(42.3061)
tensor(38.3550)
tensor(70.6300)
tensor(54.1062)
tensor(52.7224)
tensor(54.6818)
tensor(56.9151)
tensor(35.9119)
tensor(65.7046)
tensor(35.8831)
tensor(78.5277)
tensor(49.0616)
tensor(76.5194)
tensor(71.8071)
tensor(46.2532)
tensor(39.0149)
tensor(47.7201)
tensor(48.6626)
tensor(55.3286)
tensor(71.9575)
tensor(3

...

tensor(62.8117)
tensor(56.9482)
tensor(62.5960)
tensor(41.0081)
tensor(62.2524)
tensor(55.5746)
tensor(49.0730)
tensor(66.8537)
tensor(37.4092)
tensor(46.9800)
tensor(31.7862)
tensor(44.1443)
tensor(47.4512)
tensor(61.5470)
tensor(58.4598)
tensor(58.2612)
tensor(47.8131)
tensor(52.9229)
tensor(53.3712)
tensor(61.4461)
tensor(44.1089)
tensor(46.8409)
tensor(41.5755)
tensor(45.8061)
tensor(51.3594)
tensor(66.0365)
tensor(68.8889)
tensor(51.7711)
tensor(41.5522)
tensor(40.1332)
tensor(52.6476)
tensor(47.1050)
tensor(48.9705)
tensor(62.6901)
tensor(63.3105)
tensor(43.7854)
tensor(61.8803)
tensor(53.2813)
tensor(50.1458)
tensor(54.8476)
tensor(56.9502)
tensor(42.5716)
tensor(52.7428)
tensor(50.4313)
tensor(47.9481)
tensor(56.9368)
tensor(67.6966)
tensor(39.3957)
tensor(47.1871)
tensor(73.8743)
tensor(50.0458)
tensor(48.5380)
tensor(43.3813)
tensor(55.3651)
tensor(54.3307)
tensor(59.7650)
tensor(52.8315)
tensor(68.6510)
tensor(55.1453)
tensor(64.6869)
tensor(39.7144)
tensor(42.2729)
tensor(3

tensor(42.4998)
tensor(54.9668)
tensor(51.4284)
tensor(41.0489)
tensor(41.5089)
tensor(42.2618)
tensor(49.2890)
tensor(36.2786)
tensor(33.4878)
tensor(51.2902)
tensor(26.6518)
tensor(44.1292)
tensor(40.8924)
tensor(50.4801)
tensor(70.9379)
tensor(56.8984)
tensor(60.5987)
tensor(53.6048)
tensor(37.3714)
tensor(73.4825)
tensor(45.3978)
tensor(41.1332)
tensor(41.5354)
tensor(59.0262)
tensor(48.1147)
tensor(60.5704)
tensor(49.1703)
tensor(45.0215)
tensor(32.3703)
tensor(76.5335)
tensor(47.2961)
tensor(61.1218)
tensor(68.8928)
tensor(55.5751)
tensor(51.2254)
tensor(47.0208)
tensor(42.7218)
tensor(48.2815)
tensor(65.3913)
tensor(60.3951)
tensor(63.9869)
tensor(66.0425)
tensor(61.0689)
tensor(65.6406)
tensor(58.3929)
tensor(51.9280)
tensor(64.3721)
tensor(51.3871)
tensor(49.2570)
tensor(56.8531)
tensor(47.8561)
tensor(43.7504)
tensor(48.5119)
tensor(40.7925)
tensor(68.5278)
tensor(52.8769)
tensor(58.9767)
tensor(32.1037)
tensor(48.3977)
tensor(49.1652)
tensor(59.4285)
tensor(25.7019)
tensor(4

tensor(58.6969)
tensor(58.5137)
tensor(49.8319)
tensor(53.3070)
tensor(63.9609)
tensor(53.1932)
tensor(61.5039)
tensor(48.8869)
tensor(63.1682)
tensor(57.2943)
tensor(40.4071)
tensor(50.6415)
tensor(71.7132)
tensor(49.9858)
tensor(41.1312)
tensor(62.4162)
tensor(42.7803)
tensor(35.5875)
tensor(72.7405)
tensor(58.3864)
tensor(46.6985)
tensor(64.2601)
tensor(48.4563)
tensor(40.5122)
tensor(63.3899)
tensor(59.7822)
tensor(73.0023)
tensor(41.6040)
tensor(32.5686)
tensor(39.6417)
tensor(51.5454)
tensor(59.3613)
tensor(52.6690)
tensor(63.8802)
tensor(52.4096)
tensor(63.8348)
tensor(67.6767)
tensor(66.4520)
tensor(49.1890)
tensor(57.3010)
tensor(43.6127)
tensor(45.4747)
tensor(25.6230)
tensor(59.0666)
tensor(56.2146)
tensor(48.9866)
tensor(61.7392)
tensor(56.1321)
tensor(53.1819)
tensor(54.2845)
tensor(48.6919)
tensor(52.5775)
tensor(52.7392)
tensor(40.3992)
tensor(42.5402)
tensor(65.5403)
tensor(54.6808)
tensor(63.5925)
tensor(48.8002)
tensor(52.8501)
tensor(50.9853)
tensor(46.4265)
tensor(4

tensor(39.6415)
tensor(48.6521)
tensor(31.3139)
tensor(44.7209)
tensor(46.7795)
tensor(58.0347)
tensor(40.2376)
tensor(30.2084)
tensor(52.3666)
tensor(53.0112)
tensor(61.7107)
tensor(49.5777)
tensor(46.9372)
tensor(70.8678)
tensor(20.2025)
tensor(49.8188)
tensor(60.5503)
tensor(37.4390)
tensor(45.7921)
tensor(48.0481)
tensor(49.2197)
tensor(80.5480)
tensor(40.2107)
tensor(41.4952)
tensor(47.1166)
tensor(50.2555)
tensor(43.7524)
tensor(67.7253)
tensor(56.4026)
tensor(43.6143)
tensor(68.1823)
tensor(51.7141)
tensor(63.2912)
tensor(48.7158)
tensor(42.8864)
tensor(47.1796)
tensor(42.9172)
tensor(45.0129)
tensor(48.8024)
tensor(45.4227)
tensor(36.7551)
tensor(33.4745)
tensor(27.8930)
tensor(52.8097)
tensor(54.4103)
tensor(39.7041)
tensor(44.4892)
tensor(64.0256)
tensor(52.8399)
tensor(33.9902)
tensor(71.3130)
tensor(36.2361)
tensor(55.7698)
tensor(57.6277)
tensor(67.4554)
tensor(35.0528)
tensor(47.7754)
tensor(51.2802)
tensor(49.7141)
tensor(50.6806)
tensor(41.4857)
tensor(66.1603)
tensor(4

tensor(60.4175)
tensor(38.7661)
tensor(49.5894)
tensor(54.1244)
tensor(23.4541)
tensor(64.9883)
tensor(69.4062)
tensor(39.6100)
tensor(51.3968)
tensor(58.5257)
tensor(52.2707)
tensor(33.0183)
tensor(59.2950)
tensor(67.9680)
tensor(50.3757)
tensor(45.4558)
tensor(50.2053)
tensor(40.6214)
tensor(56.1150)
tensor(44.7794)
tensor(60.5260)
tensor(36.5267)
tensor(47.6156)
tensor(40.4480)
tensor(62.2559)
tensor(57.6065)
tensor(68.7851)
tensor(61.4700)
tensor(43.2880)
tensor(62.5286)
tensor(58.1834)
tensor(48.0939)
tensor(49.2837)
tensor(46.9840)
tensor(51.7453)
tensor(52.2408)
tensor(53.0807)
tensor(39.5953)
tensor(56.8728)
tensor(55.1006)
tensor(57.8736)
tensor(65.3892)
tensor(62.6644)
tensor(48.3782)
tensor(39.1324)
tensor(63.6028)
tensor(56.5170)
tensor(55.1580)
tensor(37.8734)
tensor(60.7452)
tensor(50.4960)
tensor(53.8846)
tensor(56.4367)
tensor(79.8384)
tensor(45.6961)
tensor(45.0145)
tensor(40.9816)
tensor(70.8528)
tensor(41.8946)
tensor(61.1340)
tensor(58.1802)
tensor(58.6001)
tensor(4

In [99]:
loss

In [100]:

plt.plot(range(len(loss)),loss)

TypeError: object of type 'NoneType' has no len()

In [84]:
type(loss[0])

float

In [105]:
5117342/10**5

51.17342